In [1]:
import json
import numpy as np
import os
from pathlib import Path
from kmeans_anchors_ratios import get_optimal_anchors_ratios

In [2]:
project_name = '0509split'

In [3]:
path = Path.cwd()
json_save_path = path / f'datasets/{project_name}/annotations/instances'

In [4]:
INSTANCES_PATH = str(json_save_path)+'_train.json'
with open(INSTANCES_PATH) as f:
    instances = json.load(f)
PHI = 2  # for another efficientdet change only this, e.g. PHI = 3 for D3

input_sizes = [512, 640, 768, 896, 1024, 1280, 1280, 1536, 1536]
pyramid_levels = [5, 5, 5, 5, 5, 5, 5, 5, 6]
anchor_scale = [4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 5.0, 4.0]

coef= 2.8
scale = anchor_scale[PHI]
strides = 2 ** np.arange(3, pyramid_levels[PHI] + 3)
scales = np.array([2 ** 0/coef, 2 ** (1.0 / 3.0)/coef, 2 ** (2.0 / 3.0)/coef])

INPUT_SIZE = input_sizes[PHI]
ANCHORS_SIZES = (scale * scales * strides[:, np.newaxis]).flatten().tolist()
# ANCHORS_SIZES

In [5]:
scales

array([0.35714286, 0.4499718 , 0.56692895])

In [6]:
from kmeans_anchors_ratios import get_optimal_anchors_ratios

anchors_ratios = get_optimal_anchors_ratios(
    instances,
    anchors_sizes=ANCHORS_SIZES,
    input_size=INPUT_SIZE,
    normalizes_bboxes=True,
    num_runs=10,
    num_anchors_ratios=5,
    max_iter=300,
    iou_threshold=0.5,
    min_size=0,
    decimals=1,
    default_anchors_ratios=[(0.7, 1.4), (1.0, 1.0), (1.4, 0.7)]
)

[05/09 17:52:26] Starting the calculation of the optimal anchors ratios
[05/09 17:52:26] Extracting and preprocessing bounding boxes
[05/09 17:52:26] Discarding 0 bounding boxes with size lower or equal to 0


[05/09 17:52:26] K-Means (10 runs): 100%|███████████████| 10/10 [00:07<00:00,  1.34it/s]

	Runs avg. IoU: 83.75% ± 0.01% (mean ± std. dev. of 10 runs, 0 skipped)
	Avg. IoU between bboxes and their most similar anchors after norm. them to make their area equal (only ratios matter): 83.75%


[05/09 17:52:35] Default anchors ratios: [(0.7, 1.4), (1.0, 1.0), (1.4, 0.7)]
	Avg. IoU between bboxes and their most similar default anchors, no norm. (both ratios and sizes matter): 72.96%
	Num. bboxes without similar default anchors (IoU < 0.5):  5623/38448 (14.62%)
[05/09 17:52:35] K-Means anchors ratios: [(0.7, 1.5), (0.9, 1.1), (1.2, 0.8), (1.7, 0.6), (3.3, 0.3)]
	Avg. IoU between bboxes and their most similar K-Means anchors, no norm. (both ratios and sizes matter): 80.54%
	Num. bboxes without similar K-Means anchors (IoU < 0.5):  640/38448 (1.66%)
[05/09 17:52:36] K-Means anchors have an IoU < 50% with bboxes in 12.96% less cases than the default anchors, you should consider to use them


In [ ]:

coef= [i for i in np.arange(1,4,0.1)]
scale = anchor_scale[PHI]
strides = 2 ** np.arange(3, pyramid_levels[PHI] + 3)
INPUT_SIZE = input_sizes[PHI]
for c in coef:
    scales = np.array([2 ** 0/coef, 2 ** (1.0 / 3.0)/coef, 2 ** (2.0 / 3.0)/coef])
    ANCHORS_SIZES = (scale * scales * strides[:, np.newaxis]).flatten().tolist()
    anchors_ratios = get_optimal_anchors_ratios(
    instances,
    anchors_sizes=ANCHORS_SIZES,
    input_size=INPUT_SIZE,
    normalizes_bboxes=True,
    num_runs=3,
    num_anchors_ratios=3,
    max_iter=300,
    iou_threshold=0.5,
    min_size=0,
    decimals=1,
    default_anchors_ratios=[(0.7, 1.4), (1.0, 1.0), (1.4, 0.7)]
)

In [21]:
from kmeans_anchors_ratios import generate_anchors_given_ratios_and_sizes


anchors = generate_anchors_given_ratios_and_sizes(anchors_ratios, ANCHORS_SIZES)
print("Anchors:")
print(anchors)

Anchors:
[[  4.8         12.8       ]
 [  9.6          6.4       ]
 [ 13.6          4.8       ]
 [  6.04762104  16.12698944]
 [ 12.09524208   8.06349472]
 [ 17.13492628   6.04762104]
 [  7.61952505  20.31873347]
 [ 15.2390501   10.15936673]
 [ 21.58865431   7.61952505]
 [  9.6         25.6       ]
 [ 19.2         12.8       ]
 [ 27.2          9.6       ]
 [ 12.09524208  32.25397888]
 [ 24.19048416  16.12698944]
 [ 34.26985256  12.09524208]
 [ 15.2390501   40.63746693]
 [ 30.4781002   20.31873347]
 [ 43.17730861  15.2390501 ]
 [ 19.2         51.2       ]
 [ 38.4         25.6       ]
 [ 54.4         19.2       ]
 [ 24.19048416  64.50795775]
 [ 48.38096832  32.25397888]
 [ 68.53970511  24.19048416]
 [ 30.4781002   81.27493386]
 [ 60.9562004   40.63746693]
 [ 86.35461723  30.4781002 ]
 [ 38.4        102.4       ]
 [ 76.8         51.2       ]
 [108.8         38.4       ]
 [ 48.38096832 129.01591551]
 [ 96.76193663  64.50795775]
 [137.07941023  48.38096832]
 [ 60.9562004  162.54986772]
 [121

In [22]:
from kmeans_anchors_ratios import get_bboxes_adapted_to_input_size


resized_bboxes = get_bboxes_adapted_to_input_size(instances, input_size=INPUT_SIZE)
resized_bboxes = resized_bboxes[resized_bboxes.prod(axis=1) > 0]  # remove 0 size
print("Bounding boxes adapted to the input size (first 5):")
print(resized_bboxes[:5])

Bounding boxes adapted to the input size (first 5):
[[115.2  30. ]
 [  9.6   6. ]
 [ 16.8  55.2]
 [ 15.6  26.4]
 [ 12.   33.6]]


In [23]:
from kmeans_anchors_ratios import average_iou


avg_iou = average_iou(resized_bboxes, anchors)
print(f"Avg. IoU: {100 * avg_iou:.2f}%")

Avg. IoU: 77.23%


In [24]:
from kmeans_anchors_ratios import get_annotations_without_similar_anchors


annotations = get_annotations_without_similar_anchors(
    instances,
    anchors_ratios,
    anchors_sizes=ANCHORS_SIZES,
    input_size=INPUT_SIZE,
    iou_threshold=0.5,
    min_size=0,
)

bboxes = [ann["bbox"][-2:] for ann in annotations]  # widths and heights
print("Bounding boxes without similar anchors (first 5):")
print(bboxes[:5])

instances_without_similar_anchors = instances.copy()
instances_without_similar_anchors["annotations"] = annotations
resized_bboxes = get_bboxes_adapted_to_input_size(instances_without_similar_anchors, input_size=INPUT_SIZE)
print("Bounding boxes without similar anchors adapted to the input size (first 5):")
print(resized_bboxes[:5])

Bounding boxes without similar anchors (first 5):
[[13, 6], [908, 44], [771, 53], [145, 16], [118, 15]]
Bounding boxes without similar anchors adapted to the input size (first 5):
[[  7.8   3.6]
 [544.8  26.4]
 [462.6  31.8]
 [ 87.    9.6]
 [ 70.8   9. ]]


In [25]:
annotations

[{'segmentation': [[826, 453, 826, 459, 839, 459, 839, 453]],
  'area': 78,
  'iscrowd': 0,
  'ignore': 0,
  'image_id': 26,
  'bbox': [826, 453, 13, 6],
  'category_id': 1,
  'id': 50},
 {'segmentation': [[339, 574, 339, 618, 1247, 618, 1247, 574]],
  'area': 39952,
  'iscrowd': 0,
  'ignore': 0,
  'image_id': 44,
  'bbox': [339, 574, 908, 44],
  'category_id': 3,
  'id': 89},
 {'segmentation': [[365, 612, 365, 665, 1136, 665, 1136, 612]],
  'area': 40863,
  'iscrowd': 0,
  'ignore': 0,
  'image_id': 60,
  'bbox': [365, 612, 771, 53],
  'category_id': 3,
  'id': 116},
 {'segmentation': [[947, 540, 947, 556, 1092, 556, 1092, 540]],
  'area': 2320,
  'iscrowd': 0,
  'ignore': 0,
  'image_id': 108,
  'bbox': [947, 540, 145, 16],
  'category_id': 3,
  'id': 223},
 {'segmentation': [[759, 644, 759, 659, 877, 659, 877, 644]],
  'area': 1770,
  'iscrowd': 0,
  'ignore': 0,
  'image_id': 129,
  'bbox': [759, 644, 118, 15],
  'category_id': 3,
  'id': 262},
 {'segmentation': [[599, 625, 599, 6